<a href="https://colab.research.google.com/github/Dimildizio/system_design/blob/main/create_translations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
%%capture
!pip install transformers sentencepiece googletrans==4.0.0-rc1
#set a PARTICULAR version of google trans omg
#Marian model requires sentence piece

In [68]:
from googletrans import Translator
import pandas as pd
import sentencepiece

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, MarianTokenizer, MarianMTModel
from typing import List

In [23]:
access_token =''

## Upload text

In [17]:
df = pd.read_excel('eval_data.xlsx')
df.head()

,ru,en_expert,en_auto
0,На основе данной модели реализован эксперимент...,An experimental method is implemented based on...,Based on this model is implemented experimenta...
1,Прецизионное уточнение атомной структуры минер...,Accurate crystal structure refinement of natro...,Precise refinement of atomic structure mineral...
2,Здесь и далее: точки – экспериментальные значе...,"Hereinafter, circles are experimental data, so...","Hereinafter,: point - experimental values, sol..."
3,Об образовании метастабильных фаз при кристалл...,On the formation of metastable phases during c...,About formation metastable phases at crystalli...
4,Магнитоэлектрический эффект в трехслойных асим...,Magnetoelectric Effect in Three-Layer Asymmetr...,Magnetoelectric effect in a three-layer asymme...


## Google translate the text

In [ ]:
translator = Translator()

In [21]:
def g_trans(sent: str) -> str:
  return translator.translate(sent.lower(), src='ru', dest='en').text

In [22]:
df['en_google'] = df['ru'].apply(g_trans)

### NLLB model

In [57]:
%%capture
rus_tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang="rus_Cyrl", token=access_token)

In [58]:
%%capture
nnlb_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", token=access_token)

In [59]:
class NLLB_Translate:
  def __init__(self, model, tokenizer, target_lang: str = 'eng_Latn', sent_len: int = 300):
    self.model = model
    self.tokenizer = tokenizer
    self.to_lang = target_lang
    self.max_length = sent_len

  def tokenize(self, sent: str):
    return self.tokenizer(sent, return_tensors='pt')

  def translate(self, inputs):
    return self.model.generate(
      **inputs, forced_bos_token_id=self.tokenizer.lang_code_to_id[self.to_lang],
      max_length=self.max_length)

  def get_decoded(self, toks) -> List:
    return self.tokenizer.batch_decode(toks, skip_special_tokens=True)

  def process_sentence(self, sent: str):
    tokens = self.tokenize(sent)
    translated = self.translate(tokens)
    result = self.get_decoded(translated)
    return result

In [60]:
nllb_trans_model = NLLB_Translate(model, rus_tokenizer)

In [48]:
sentence = nllb_trans_model.process_sentence(df['ru'][0])
sentence[0]

In [89]:
def nllb_trans(sent: str) -> str:
  return nllb_trans_model.process_sentence(sent)[0]

In [90]:
df1=df.copy()

In [92]:
df1['en_nllb'] = df1['ru'].apply(nllb_trans)